#Что такое AXON

`AXON` это нотация для сериализованного представления объектов, документов и данных в текстовой форме.

In [1]:
import axon

#### AXON это несколько улучшенный вариант JSON

`JSON` *всегда* заключает имена в кавычки, `AXON` *позволяет не заключать* в кавычки имена, которые являются *идентификаторами*. Также в `AXON` *опускаются* разделительные *запятые*, используются только пробельные символы для разделения значений. 
В результате получается более компактное представление.

Для сравнения:

**`JSON`**
```
{ "name": "Alex",
  "age": 32,
  "email": "mail@example.com"}
["Alex", 32, "mail@example.com"]
```
**`AXON`**
```
{ name: "Alex"
  age: 32
  email: "mail@example.com"}
["Alex" 32 "mail@example.com"]
```

`AXON` имеет синтаксиc для представления некоторых других атомарных типов: 

* даты

```
2010-12-31

```

* времени

```
12:30
12:30:15
12:30+03:00
12:30:15-04:30
```

* даты и времени

```
2010-12-31T12:30
2010-12-31T12:30:05.0125
2010-12-31T12:30+04:00
2010-12-31T12:30:05.0123-04:00
```

* десятичных чисел

```
1D 123456789D
3.14D 1.23e-6D
```

`AXON` позволяет определять *метки* неатомарных значений и использовать их в качестве *ссылок*. Это позволяет не создавать копии многократно используемых неатомарных значений при сериализации и десериализации.

Например:

``` javascript
[ { prev: &a (2012-12-30 10:00)
    next: &c (2012-01-01 12:00) }
  { prev: &b (2012-12-31 13:00)
    next: *a }
  { prev: *c
    next: *b } ]
```

Здесь определение метки значения имеет префикс `&` (`&a &b &c`) и оно предваряет значение для которого она определена, а ссылка при помощи метки имеет префикс `*` (`*a *b *c`).

#### AXON содержит упрощенный вариант Infoset XML, но в более компактной нотации

Рассмотрим пример `XML` представления:

``` xml
<person>
  <name>John Smith</name>
  <age>25</age>
  <address type="home">
     <street>21 2nd Street</street>
     <city>New York</city>
     <state>NY</state>
  </address>
  <address type="current">
     <street>1410 NE Campus Parkway</street>
     <city>Seattle</city>
     <state>WA</state>
  </address>
  <phone type="home">212-555-1234</phone>
  <phone type="fax">646-555-4567</phone>
</person>
```
Его аналог в формате `AXON` имеет вид:

``` javascript
person {
  name {"John Smith"}
  age {25}
  address { 
     type: "home"
     street {"21 2nd Street"}
     city {"New York"}
     state {"NY"}
  }
  address {
     type: "current"
     street {"1410 NE Campus Parkway"}
     city {"Seattle"}
     state {"WA"}
  }
  phone {type:"home" "212-555-1234"}
  phone {type:"fax" "646-555-4567"}
}
```

Представление в формате `AXON` можно построить из формата `XML` за 5 шагов:

1. Заменить `<tag>` на `tag {`
2. Заменить `</tag>` на `}`
3. Заменить `attr=value` на `attr: value`
4. Текст внутри элементов заключить в двойные кавычки (`"`)
5. Удалить символ запятой (`,`) или заменить его на один пробел

Результат такого преобразования структурно идентичен первоначальному `XML` документу. По-существу это синтаксически более компактная форма представления `XML` документа.

Этот пример демонстрирует идею упрощенного представления `XML` документа при помощи `AXON`.

Для сравнения также приведем представление в `AXON` с форматированием сложных элементов без {}:

``` javascript
person
  name {"John Smith"}
  age {25}
  address
     type: "home"
     street {"21 2nd Street"}
     city {"New York"}
     state {"NY"}
  address
     type: "current"
     street {"1410 NE Campus Parkway"}
     city {"Seattle"}
     state {"WA"}
  phone
      type: "home"
      "212-555-1234"
  phone
      type:"fax"
      "646-555-4567"
```

Это представление получается из предыдущего удалением всех символов { и }, пустых строк.

#### AXON расширяет возможности XML и JSON

В `XML` атрибуты могут иметь только простые значения, в `AXON` значением атрибута может любое значение (как и в `JSON`). Кроме того простые значения имеют тип (*текст* в формате `unicode`, *число*, *десятичное число*, *дата* и *время*, *массив байтов* в кодировке *base64*). `AXON` можно рассматривать как расширение `JSON` в том смысле, что объекты могут именованными, так же как и элементы `XML` являются именованными.

Например:

``` javascript
person
  name: "John Smith"
  age: 25
  burn: 1975-10-21
  locations: [
    address
      type: "home"
      street: "21 2nd Street"
      city: "New York"
      state: "NY"
    address
      type: "current"
      street: "1410 NE Campus Parkway"
      city: "Seattle"
      state: "WA"
  ]
  contacts: [
    phone 
      type: "home"
      "212-555-1234"
    phone
      type: "fax" 
      "646-555-4567"
    email
      type: "personal"
      "mail@example.com"
  ]
```


В `JSON` есть одно неудобство, связанное с представлением нерегулярных структур, в которых существенен порядок частей. В таких структурах доступ к элементам осуществляется в результате последовательного поиска по имени, а не в результате "прямого" доступа по имени.

В качестве примера рассмотрим структурированный документ в формате `XML`:

``` xml
<section title="Title">
<par style="normal">paragraph</par>
<enumerate style="enum">
<item>item text</item>
<item>item text</item>
</enumerate>
<par style="normal">paragraph</par>
<itemize style="itemize">
<item>item text</item>
<item>item text</item>
</itemize>
<par style="normal">paragraph</par>
</section>
```

Непосредственно в `JSON` этот документ не транслируется из-за важности порядка и повторяемости элементов. Один вариант трансляции, который эмулирует последовательность именованных элементов имеет вид:

``` javascript
{
  "tag": "section",
  "attrs": {"title": "Title"},
  "nodes": [
     { "tag": "par", "style":"normal", "text":"paragraph"},
     { "tag":"enumerate", "style": "enumerate",
       "nodes": [
         { "tag":"item", "text":"item text"},
         { "tag":"item", "text":"item text"}
       ]
     },
     { "tag": "par", "style":"normal", "text":"paragraph"},
     { "tag":"itemize",
       "nodes": [
         { "tag":"item", "text":"item text"},
         { "tag":"item", "text":"item text"}
       ]
     },
     { "tag": "par", "style":"normal", "text":"paragraph"}     
  ]
}
```

В `AXON` можно транслировать такие структуры "один в один":

``` javascript
section
  title: "Title"
  par 
    style: "normal"
    "paragraph"
  enumerate
    style: "enum"
    item { "item text" }
    item { "item text" }
  par
    style: "normal"  
    "paragraph"
  itemize
    style: "itemize"
    item { "Item text" }
    item { "Item text" }
  par
    style: "normal"  
    "paragraph"
```


`AXON` имеет возможность эмулировать "пространства имен". Например:
    
``` javascript
{ xs.type: "element"
  xs.name: "person"
  py.type: "Person"
}

```

### AXON поддерживает форматирование в стиле YAML

Привлекательной стороной `YAML` является формат представления в стиле `wiki`. `AXON` также поддерживает подобный стиль форматирования.
Например:

* форматирование без {} (`YAML`-стиль)

``` javascript
person
  name: "Alex"
  age: 25
```

* форматирование с {} и отступами (`C/JSON`-стиль)

``` javascript
person {
  name: "Alex"
  age: 25}
```

* компактный формат

``` javascript
person{name:"Alex" age:25}
```

### AXON может представлять серию объектов

Одно из ограничений `JSON` связано с тем, что он представляет единственный корневой объект. Напротив, `AXON` представляет серию объектов или серию пар `ключ`:`объект`. Например:

* серия объектов

``` javascript
{ name: "Alex"
  age: 32 }
{ name: "Michael"
  age: 28 }
{ name: "Nick"
  age: 19 }
```

* серия объектов с ключами

``` javascript
alex: {
    message: "Hello"
    datetime: 2015-07-12T12:32:35
}
michael: {
    message: "How are you"
    datetime: 2015-07-12T12:32:35
}

```